In [2]:
# Import Libraries 
import hakom
import json
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import getpass
import urllib.request, json 
import pandas as pd
from datetime import datetime, timedelta
#teständerung

## Initialize Python Connector

In [3]:
api_pwd = getpass.getpass()


In [4]:
auth_params = hakom.authorization.AuthParams(
    username = 'aeneas',
    password = api_pwd,
)

service_config = hakom.service.ServiceConfig(
    base_url="https://fehostedwebapp.azurewebsites.net/MAIN/api/",
    auth_params=auth_params,
    repository='ZAMS'
)

hakom.initialize(service_config)


## Define Functions

In [5]:
def parse_inmet(station_str):
    """ Takes inmet station number as input to download data from beginning of 2018 to end 2022. 
    Returns all data in a pandas-df.
    Expexted shape is (43824, 27)  """
    data =[]
    for year in range(2018,2023):
        date_begin = '{}-01-01'.format(year)
        date_end = '{}-12-31'.format(year)
        url_str = 'https://apitempo.inmet.gov.br/estacao/{}/{}/{}'.format(date_begin, date_end, str(station_str))
        print(url_str)
      
        with urllib.request.urlopen(url_str) as url:
            temp = pd.read_json(url.read().decode())
        data.append(temp)
    data = pd.concat(data)
    return data


def timeframe_inmet(data):
    """Takes pandas-df, adds timestamp as index (hour). 
    Returns whole data frame or only the temperature column """
    data['timestamp'] = pd.to_datetime(data['DT_MEDICAO']) + pd.to_timedelta(data['HR_MEDICAO'] // 100, unit='h')
    data = data.set_index('timestamp')
    data = data.asfreq('h')
    
    df_temp = data[['TEM_INS']]
    df_wdir = data[['VEN_DIR']]
    df_wspeed = data[['VEN_VEL']]
    return data, df_temp


def inmet_TEMP_into_TSM_format(df):
    """Takes a temperature df and transforms it into HAKOM data format.
    Returns HAKOM data format, ready to load into DB """
    df = hakom.pandas_interop.create_timestamp_colum_from_index(df)
    df['flag'] = 9
    one_hour_interval = hakom.types.Interval(hakom.types.Intervals.Hour)
    unit_label = '°C'
    tsdc = hakom.pandas_interop.to_hakom_tsdata_collection(
        df.dropna(),
        interval=one_hour_interval,
        unit=unit_label,
        dt_flag_value_order=[2,1,3]
    )
    return tsdc

## Go for the loop

In [30]:
# Define stations
station_str = ['A521', 'A707', 'A652','A002', 'A747', 'A612']
#station_str = ['A707']

# Define TSM TS ID's
TSM_IDs = ['2500', '2516', '2514', '2513', '2517', '2515']
#TSM_IDs = [2516]

zib = zip(station_str, TSM_IDs)

for (station, ID) in zib:
    a = parse_inmet(station)
    _,b = timeframe_inmet(a)
    c = inmet_TEMP_into_TSM_format(b)
    
    hakom.repositories.timeseries.data.save(ID, c)




https://apitempo.inmet.gov.br/estacao/2018-01-01/2018-12-31/A521
https://apitempo.inmet.gov.br/estacao/2019-01-01/2019-12-31/A521
https://apitempo.inmet.gov.br/estacao/2020-01-01/2020-12-31/A521
https://apitempo.inmet.gov.br/estacao/2021-01-01/2021-12-31/A521
https://apitempo.inmet.gov.br/estacao/2022-01-01/2022-12-31/A521
https://apitempo.inmet.gov.br/estacao/2018-01-01/2018-12-31/A707
https://apitempo.inmet.gov.br/estacao/2019-01-01/2019-12-31/A707
https://apitempo.inmet.gov.br/estacao/2020-01-01/2020-12-31/A707
https://apitempo.inmet.gov.br/estacao/2021-01-01/2021-12-31/A707
https://apitempo.inmet.gov.br/estacao/2022-01-01/2022-12-31/A707
https://apitempo.inmet.gov.br/estacao/2018-01-01/2018-12-31/A652
https://apitempo.inmet.gov.br/estacao/2019-01-01/2019-12-31/A652
https://apitempo.inmet.gov.br/estacao/2020-01-01/2020-12-31/A652
https://apitempo.inmet.gov.br/estacao/2021-01-01/2021-12-31/A652
https://apitempo.inmet.gov.br/estacao/2022-01-01/2022-12-31/A652
https://apitempo.inmet.go

UndefinedError: <!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Strict//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-strict.dtd">
<html xmlns="http://www.w3.org/1999/xhtml">
<head>
<meta http-equiv="Content-Type" content="text/html; charset=iso-8859-1"/>
<title>502 - Web server received an invalid response while acting as a gateway or proxy server.</title>
<style type="text/css">
<!--
body{margin:0;font-size:.7em;font-family:Verdana, Arial, Helvetica, sans-serif;background:#EEEEEE;}
fieldset{padding:0 15px 10px 15px;} 
h1{font-size:2.4em;margin:0;color:#FFF;}
h2{font-size:1.7em;margin:0;color:#CC0000;} 
h3{font-size:1.2em;margin:10px 0 0 0;color:#000000;} 
#header{width:96%;margin:0 0 0 0;padding:6px 2% 6px 2%;font-family:"trebuchet MS", Verdana, sans-serif;color:#FFF;
background-color:#555555;}
#content{margin:0 0 0 2%;position:relative;}
.content-container{background:#FFF;width:96%;margin-top:8px;padding:10px;position:relative;}
-->
</style>
</head>
<body>
<div id="he

range(0, 1)